# Retail Sales Forecasting

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas.tseries.offsets import DateOffset

## Preparing the Data

In [11]:
# Load the dataset with a different encoding
file_path = 'data/store-sales-forecasting.csv'
data = pd.read_csv(file_path, encoding='latin1')

data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
3,6,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
4,11,CA-2014-115812,6/9/2014,6/14/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-TA-10001539,Furniture,Tables,Chromcraft Rectangular Conference Tables,1706.1840,9,0.20,85.3092


### Feature Optimization

Features to Keep:
1. Order Date: To determine the sales month.
2. Sales: To calculate the total monthly sales.
Additionally, consider keeping features that might provide insights into sales trends, such as:
3. Segment: To analyze sales trends across different customer segments.
4. Ship Mode: To see if different shipping modes impact sales.
5. Region: To understand regional sales trends.

Features to Remove:
1. Row ID: Only serves as a unique identifier and does not contribute to forecasting.
2. Order ID: Similar to Row ID, it doesn't contribute directly to forecasting.
3. Ship Date: Not directly relevant to sales forecasting.
4. Customer ID: Individual customer tracking is less relevant for aggregate monthly sales.
5. Customer Name: Personal details not necessary for sales forecasting.
6. Country: If all data points are from the same country, this is redundant.
7. City: Can be too granular for monthly forecasting; Region is more relevant.
8. State: Similar to City, might be too granular if Region is included.
9. Postal Code: Too granular.
10. Product ID: Detailed product information may not be necessary for overall sales trends.
11. Product Name: Not needed if Product ID is excluded.
12. Quantity: Total sales amount is more relevant.
13. Discount: Could be removed unless you want to analyze its effect on sales.
14. Profit: Not directly related to the total sales amount; can be analyzed separately.


In [12]:
# Remove unnecessary columns
columns_to_remove = [
    'Row ID', 'Order ID', 'Ship Date', 'Customer ID', 'Customer Name',
    'Country', 'City', 'State', 'Postal Code', 'Product ID', 'Product Name',
    'Quantity', 'Discount', 'Profit'
]
data = data.drop(columns=columns_to_remove)

In [13]:
# Convert 'Order Date' to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'])

# Ensure 'Sales' is numeric
data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce')

# Drop rows with invalid 'Sales' values
data = data.dropna(subset=['Sales'])

# Extract month-year period
data['Month'] = data['Order Date'].dt.to_period('M')

In [14]:
# Aggregate monthly sales
monthly_sales = data.groupby('Month').agg({
    'Sales': 'sum',
    'Segment': 'first',
    'Ship Mode': 'first',
    'Region': 'first'
}).reset_index()

In [15]:
# Convert 'Month' to string for plotting
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

# Extract additional time-based features
monthly_sales['Year'] = pd.to_datetime(monthly_sales['Month']).dt.year
monthly_sales['Month_Num'] = pd.to_datetime(monthly_sales['Month']).dt.month

# Encode categorical features
monthly_sales_encoded = pd.get_dummies(monthly_sales, columns=['Segment', 'Ship Mode', 'Region'])

In [16]:
# Define features and target variable
X = monthly_sales_encoded.drop(columns=['Sales', 'Month'])
y = monthly_sales_encoded['Sales']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Train and evaluate with default parameters
rf_model_default = RandomForestRegressor(random_state=42)
rf_model_default.fit(X_train, y_train)

y_pred_train_default = rf_model_default.predict(X_train)
y_pred_test_default = rf_model_default.predict(X_test)

In [18]:
mae_train_default = mean_absolute_error(y_train, y_pred_train_default)
mse_train_default = mean_squared_error(y_train, y_pred_train_default)
mae_test_default = mean_absolute_error(y_test, y_pred_test_default)
mse_test_default = mean_squared_error(y_test, y_pred_test_default)

print(f"Default Model - Training MAE: {mae_train_default}, Training MSE: {mse_train_default}")
print(f"Default Model - Testing MAE: {mae_test_default}, Testing MSE: {mse_test_default}")

Default Model - Training MAE: 1544.7133419210488, Training MSE: 3579592.4184629954
Default Model - Testing MAE: 2712.4808404999976, Testing MSE: 11110331.275853956


In [19]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [20]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid,
                           cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')

In [21]:
# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters from GridSearchCV: ", best_params)

Best Parameters from GridSearchCV:  {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}


In [23]:
# Get the best estimator
best_rf_model = grid_search.best_estimator_

# Train and evaluate with GridSearchCV parameters
y_pred_train_grid = best_rf_model.predict(X_train)
y_pred_test_grid = best_rf_model.predict(X_test)